# Gridworldで強化学習



## ダウンロード

まずはデモ用のパッケージ
- https://github.com/AkinoriTanaka-phys/gridworld_rl

をダウンロードします。

### Google colaboratory から試す場合

以下のセルを実行するとセッションストレージにダウンロードします：

In [ ]:
!git clone https://github.com/AkinoriTanaka-phys/gridworld_rl.git

左側のフォルダマークをクリックすると、
- `gridworld_rl`（こちらが上のセルでダウンロードしたもの）
- `sample_data`（こちらはデフォルト）

と表示されるはずです。

### 手元のPC から試す場合

notebookのpythonカーネルが以下のライブラリを持っている必要があります：
- numpy
- matplotlib
- copy

万が一これらが入っていない場合は適宜入れてください。

このnotebookの直下にダウンロードしますが、別の場所に置きたい場合は適宜修正してください。

In [ ]:
%%bash
git clone https://github.com/AkinoriTanaka-phys/gridworld_rl.git

## 読み込み

今回使うクラスをインポートしておきます：

In [ ]:
import matplotlib.pyplot as plt

import numpy as np
from gridworld_rl.src.env import GridWorld, Maze
from gridworld_rl.src.agt import You, Agent, Softmax, EpsilonGreedy, Value
from gridworld_rl.src.opt import ActorCritic, REINFORCE, MonteCarlo, SARSA, Qlearning

それぞれの詳細は`help()`で参照してみてください：

In [ ]:
help(REINFORCE)

## 環境

環境は `env.py` で定義されています。


In [ ]:
env = GridWorld(5, 5)
env.render()

### Maze環境

`GridWorld`を継承して、もう少し解くのが難しそうな環境も用意しました。

In [ ]:
env = Maze(5, 5, sparcity=1.3)
env.render()

## エージェント

エージェントは `agt.py` で定義しています。エージェントには環境 `env` を食わせます。

環境の振る舞いを調べるために、ユーザー自身をエージェントとして動かすことができます。エージェントの名前は`You`です：

In [ ]:
agt = You(env)

env.reset()
while not env.is_terminated():
    env.render()
    s = env.get_state()
    a = agt.play(s)
    sn, rn, _, _ = env.step(a)
    print(f"reward={rn}")
env.render()

### 学習エージェント

学習用のエージェントは `Agent` に適当な方策を与えて定義します。`agt.py` では
- `Softmax`（方策勾配法向き）
- `EpsilonGreedy`（それ以外向き、デフォルトε=0.1）

を定義しています。

In [ ]:
agt = Agent(env, Policy_class=Softmax)

env.reset()
while not env.is_terminated():
    env.render()
    s = env.get_state()
    a = agt.play(s)
    sn, rn, _, _ = env.step(a)
    print(f"move={env.ACTION2STR[a]}, reward={rn}")
env.render()

## 強化学習アルゴリズム

強化学習アルゴリズムは `opt.py` で定義しています。 アルゴリズムにはエージェント `agt` を食わせます。

現行のモデルは `GridWorld` や `Maze` の座標ごとに4種の行動（up, down, left, right）についてのパラメータを持っているものですが、ニューラルネットなどを用いて一般化することも可能かと思います。

In [ ]:
# ちょっと大きな迷路を作っておきます
env = Maze(15, 15, sparcity=1.3)
env.render()

## 方策勾配法

方策勾配法は
- `REINFORCE`
    - オプションとしてエージェント定義で `Value_class=Value` を指定するとベースライン込みのREINFORCEとなり、学習が安定化します。
- `ActorCritic`
    - こちらはエージェント定義で `Value_class=Value` の指定が必須です。

を用意しました。両方ともエージェント定義で `Policy_class=Softmax` の使用が想定されています。

In [ ]:
N_episode = 1000

agt = Agent(env, Policy_class=Softmax, Value_class=Value)
#opt = REINFORCE(agt, eta_p=.5, eta_v=.5, gamma=.99)
opt = ActorCritic(agt, eta_p=.5, eta_v=.5, gamma=.99)

for episode in range(N_episode):
    env.reset()
    opt.reset()
    while not env.is_terminated():
        s = env.get_state()
        a = agt.play(s)
        sn, rn, _, _ = env.step(a)
        opt.step(s, a, rn, sn)
    opt.flush() 

訓練後に解かせてみる：

In [ ]:
env.reset()
env.play(agt)
env.render(values_table=agt.policy.param.table)

## 価値反復法

価値＝$\langle r_1 + \gamma \cdot r_2 + \gamma^2 \cdot r_2 + \cdots \rangle$ の推定に基づく手法は
- `MonteCarlo`
    - デフォルトは素朴に1エピソード毎にモンテカルロ計算。理論からやや外れますが、アルゴリズム定義のオプションで `erase_history=False` とすると、訪問回数のサンプルを累積する設定になり、性能が上がります。
- `SARSA`
- `Qlearning`

を用意しました。

In [ ]:
N_episode = 1000

agt = Agent(env, Policy_class=EpsilonGreedy)
#opt = MonteCarlo(agt, gamma=.99, erase_history=False)
#opt = SARSA(agt, eta=.5, gamma=.99)
opt = Qlearning(agt, eta=.5, gamma=.99)

for episode in range(N_episode):
    env.reset()
    opt.reset()
    while not env.is_terminated():
        s = env.get_state()
        a = agt.play(s)
        sn, rn, _, _ = env.step(a)
        opt.step(s, a, rn, sn)
    opt.flush() 
    

訓練後に解かせてみる：

In [ ]:
env.reset()
env.play(agt)
env.render(values_table=agt.policy.param.table)